# Agriculture Analytics

-- To determine the health of the cultivated crop

[Hackathon Link](https://datahack.analyticsvidhya.com/contest/janatahack-machine-learning-in-agriculture/#ProblemStatement)

## Load packages and Data

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from collections import Counter

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
filepath= "/content/drive/My Drive/Agri_AV/"

In [ ]:
# Reading data files
train_df= pd.read_csv(f'{filepath}train_yaOffsB.csv')
test_df= pd.read_csv(f'{filepath}test_pFkWwen.csv')
submission_df= pd.read_csv(f'{filepath}sample_submission_O1oDc4H.csv')

## Pre- processing

In [ ]:
print(" The train data::")
train_df.head()

 The train data::


,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,F00000001,188,1,0,1,0,0.0,0,1,0
1,F00000003,209,1,0,1,0,0.0,0,2,1
2,F00000004,257,1,0,1,0,0.0,0,2,1
3,F00000005,257,1,1,1,0,0.0,0,2,1
4,F00000006,342,1,0,1,0,0.0,0,2,1


In [ ]:
print(" The test data::")
test_df.head()

 The test data::


,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season
0,F00000002,188,1,1,1,0,NaN,0,2
1,F00000007,410,1,1,1,0,0.0,0,2
2,F00000011,626,1,0,1,0,0.0,0,2
3,F00000013,731,1,0,1,0,0.0,0,2
4,F00000014,789,0,0,1,0,0.0,0,1


In [ ]:
print("Info about train data:: ")
train_df.info()

Info about train data:: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88858 entries, 0 to 88857
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       88858 non-null  object 
 1   Estimated_Insects_Count  88858 non-null  int64  
 2   Crop_Type                88858 non-null  int64  
 3   Soil_Type                88858 non-null  int64  
 4   Pesticide_Use_Category   88858 non-null  int64  
 5   Number_Doses_Week        88858 non-null  int64  
 6   Number_Weeks_Used        79858 non-null  float64
 7   Number_Weeks_Quit        88858 non-null  int64  
 8   Season                   88858 non-null  int64  
 9   Crop_Damage              88858 non-null  int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 6.8+ MB


In [ ]:
print("Info about test data:: ")
test_df.info()

Info about test data:: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59310 entries, 0 to 59309
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       59310 non-null  object 
 1   Estimated_Insects_Count  59310 non-null  int64  
 2   Crop_Type                59310 non-null  int64  
 3   Soil_Type                59310 non-null  int64  
 4   Pesticide_Use_Category   59310 non-null  int64  
 5   Number_Doses_Week        59310 non-null  int64  
 6   Number_Weeks_Used        53417 non-null  float64
 7   Number_Weeks_Quit        59310 non-null  int64  
 8   Season                   59310 non-null  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 4.1+ MB


* Only number of weeks used data has missing values, in both train and test data, which has detailed information
* Train data has 88858 rows
* Test data has 59310 rows

In [ ]:
# Concating train and test for further ease of computation
train_df['Org_Data']= 'Train'
test_df['Org_Data']= 'Test'
comb_df= train_df.append(test_df, ignore_index= True)
comb_df.tail(3)

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,Org_Data
148165,F00155941,3702,1,0,2,10,NaN,48,1,NaN,Test
148166,F00155943,3702,1,0,2,10,28.0,17,2,NaN,Test
148167,F00155944,3895,1,0,2,5,52.0,7,1,NaN,Test


In [ ]:
# Converting data types
colconv= ['Crop_Type', 'Soil_Type', 'Pesticide_Use_Category', 'Season']
comb_df[colconv]= comb_df[colconv].astype(object)

In [ ]:
print(" Missing values ratio in train data:: ")
train_df.isnull().sum()/ len(train_df)

 Missing values ratio in train data:: 


ID                         0.000000
Estimated_Insects_Count    0.000000
Crop_Type                  0.000000
Soil_Type                  0.000000
Pesticide_Use_Category     0.000000
Number_Doses_Week          0.000000
Number_Weeks_Used          0.101285
Number_Weeks_Quit          0.000000
Season                     0.000000
Crop_Damage                0.000000
Org_Data                   0.000000
dtype: float64

In [ ]:
print(" Missing values ratio in test data:: ")
test_df.isnull().sum()/ len(test_df)

 Missing values ratio in test data:: 


ID                         0.000000
Estimated_Insects_Count    0.000000
Crop_Type                  0.000000
Soil_Type                  0.000000
Pesticide_Use_Category     0.000000
Number_Doses_Week          0.000000
Number_Weeks_Used          0.099359
Number_Weeks_Quit          0.000000
Season                     0.000000
Org_Data                   0.000000
dtype: float64

In [ ]:
print(" Missing values ratio in combined data:: ")
comb_df.isnull().sum()/ len(comb_df)

 Missing values ratio in combined data:: 


ID                         0.000000
Estimated_Insects_Count    0.000000
Crop_Type                  0.000000
Soil_Type                  0.000000
Pesticide_Use_Category     0.000000
Number_Doses_Week          0.000000
Number_Weeks_Used          0.100514
Number_Weeks_Quit          0.000000
Season                     0.000000
Crop_Damage                0.400289
Org_Data                   0.000000
dtype: float64

The missing values percentage is around 10%.

If the missing values are removed, there will be loss of data. So, we will try different approaches to fill in the missing values.

### Label Encoding

In [ ]:
train_df.head(3)

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,F00000001,188,1,0,1,0,0.0,0,1,0
1,F00000003,209,1,0,1,0,0.0,0,2,1
2,F00000004,257,1,0,1,0,0.0,0,2,1


In [ ]:
def Encoding(col):
  le= LabelEncoder()
  comb_df[col]= le.fit_transform(comb_df[col])

  #catg= train_df[col].append(test_df[col])
  #train_df[col]= le.transform(train_df[col])
  #test_df[col]= le.transform(test_df[col])  

In [ ]:
Encoding('Crop_Type')
Encoding('Soil_Type')
Encoding('Pesticide_Use_Category')
Encoding('Season')

### Missing Value handling

In [ ]:
comb_df.head(3)

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,Org_Data
0,F00000001,188,1,0,1,0,0.0,0,1,0.0,Train
1,F00000003,209,1,0,1,0,0.0,0,2,1.0,Train
2,F00000004,257,1,0,1,0,0.0,0,2,1.0,Train


In [ ]:
comb_df[comb_df.Pesticide_Use_Category== 0].Number_Weeks_Used.value_counts()

0.0    1251
Name: Number_Weeks_Used, dtype: int64

In [ ]:
comb_df[( comb_df.Pesticide_Use_Category== 2 )].Number_Weeks_Quit.value_counts()

0    40495
Name: Number_Weeks_Quit, dtype: int64

In [ ]:
comb_df[( comb_df.Pesticide_Use_Category== 1 ) ].Number_Weeks_Quit.value_counts()

10    4806
5     4677
2     4633
12    4555
15    4470
1     4451
14    4397
6     4344
3     4281
13    4170
4     4158
7     4040
8     3801
20    3621
11    3580
0     3569
16    3532
18    3396
17    3356
9     3169
19    3027
21    2366
22    2362
23    1929
25    1796
24    1729
26    1481
27    1358
28    1310
30    1221
29    1196
31     900
32     846
33     645
34     578
35     510
36     419
37     380
38     243
40     217
39     189
41     143
42     128
43      80
44      53
45      53
46      38
47      30
48      17
50      10
49       5
Name: Number_Weeks_Quit, dtype: int64

* When Pesticide was never used(ie, 0), the corresponding columns of doses per week, quit weeks and number of weeks used will be zero.

* When Pesticide is currently being used(ie, 2), the corresponding columns for quit weeks will be zero.

* When Pesticide was previously used(ie, 1), the corresponding columns of doses per week, quit weeks and number of weeks used will be varying across a range of values 

In [ ]:
train_df[( train_df.Pesticide_Use_Category== 2 ) & (~train_df.Number_Weeks_Used.isna() ) ].Number_Weeks_Used.value_counts()

40.0    942
30.0    916
35.0    729
50.0    721
25.0    626
       ... 
63.0     47
64.0     42
65.0     29
66.0     17
67.0      8
Name: Number_Weeks_Used, Length: 63, dtype: int64

In [ ]:
print(" Mean: ", train_df[( train_df.Pesticide_Use_Category== 2 ) ].Number_Weeks_Used.mean())
print(" Median: ", train_df[( train_df.Pesticide_Use_Category== 2 ) ].Number_Weeks_Used.median())
print(" Mode: ", train_df[( train_df.Pesticide_Use_Category== 2 ) ].Number_Weeks_Used.mode()[0])

 Mean:  26.616494845360826
 Median:  25.0
 Mode:  20.0


In [ ]:
print(" Mean: ", test_df[( test_df.Pesticide_Use_Category== 2 ) ].Number_Weeks_Used.mean())
print(" Median: ", test_df[(test_df.Pesticide_Use_Category== 2 ) ].Number_Weeks_Used.median())
print(" Mode: ", test_df[(test_df.Pesticide_Use_Category== 2 ) ].Number_Weeks_Used.mode()[0])

 Mean:  26.711707240622715
 Median:  26.0
 Mode:  20.0


From mean, median and mode values of the dataframe, segregating on the different pesticide category, we see that the distribution is quite close to normal for the number of weeks used.

In [ ]:
print(" Mean: ", comb_df.Number_Weeks_Used.mean())
print(" Median: ", comb_df.Number_Weeks_Used.median())
print(" Mode: ", comb_df.Number_Weeks_Used.mode()[0])

 Mean:  28.62397004683313
 Median:  28.0
 Mode:  20.0


We tried to fill up the missing values by mean, median and mode of the number of weeks used, based on segments of pesticide category. We also tried filling up missing values in number of weeks column by the mean value of the whole column.

But by far, median of segments gave the better result. So we will go ahead with that method.

In [ ]:
train_org= train_df.copy(deep= True)
test_org= test_df.copy(deep= True)

In [ ]:
train_df[( train_df.Pesticide_Use_Category== 1 ) & ( train_df.Number_Weeks_Used.isna() )].head(3)

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
707,F00001273,169,0,0,1,30,NaN,1,1,0
708,F00001275,232,0,0,1,20,NaN,9,1,0
715,F00001289,342,0,0,1,20,NaN,3,1,0


In [ ]:
test_df[( test_df.Pesticide_Use_Category== 1 ) & ( test_df.Number_Weeks_Used.isna() )].head(3)

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Org_Data
0,F00000002,188,1,1,1,0,NaN,0,2,Test
6,F00000016,851,1,0,1,0,NaN,0,3,Test
11,F00000031,1898,0,1,1,0,NaN,0,2,Test


In [ ]:
'''# if done separately
train_df["Number_Weeks_Used"]= train_df.groupby("Pesticide_Use_Category")['Number_Weeks_Used'].transform(lambda x: x.fillna(x.median() ))

# Using the same technique in test data
test_df["Number_Weeks_Used"] = test_df.groupby("Pesticide_Use_Category")['Number_Weeks_Used'].transform(lambda x: x.fillna(x.median() ) )'''

In [ ]:
comb_org= comb_df.copy(deep= True)

In [ ]:
comb_df["Number_Weeks_Used"]= comb_df.groupby(["Org_Data", "Pesticide_Use_Category"])['Number_Weeks_Used'].transform(lambda x:
                                                                                                                     x.fillna(x.median() ))

In [ ]:
comb_df.columns

Index(['ID', 'Estimated_Insects_Count', 'Crop_Type', 'Soil_Type',
       'Pesticide_Use_Category', 'Number_Doses_Week', 'Number_Weeks_Used',
       'Number_Weeks_Quit', 'Season', 'Crop_Damage', 'Org_Data'],
      dtype='object')

In [ ]:
pes_wt= {0: 0.5, 1: 1, 2: 1.5}
comb_df["PestWt"]=  comb_df.Pesticide_Use_Category.map(pes_wt)

In [ ]:
comb_df["Pesticide_Use_Amt"]= comb_df.Number_Doses_Week* comb_df.Number_Weeks_Used
comb_df.head(3)

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,Org_Data,PestWt,Pesticide_Use_Amt
0,F00000001,188,1,0,0,0,0.0,0,0,0.0,Train,0.5,0.0
1,F00000003,209,1,0,0,0,0.0,0,1,1.0,Train,0.5,0.0
2,F00000004,257,1,0,0,0,0.0,0,1,1.0,Train,0.5,0.0


In [ ]:
comb_df["Crop_Seasn_Soil"]= comb_df.Crop_Type* comb_df.Soil_Type* comb_df.Season

In [ ]:
comb_df.Crop_Seasn_Soil.value_counts()

0    141424
1      4785
2      1959
Name: Crop_Seasn_Soil, dtype: int64

### Binning values

In [ ]:
'''train_df['Insect_Bins']= pd.cut(train_df.Estimated_Insects_Count, bins= 7)
train_df['Dose_Bins']= pd.cut(train_df.Number_Doses_Week, bins= 5)
train_df['Use_Bins']= pd.cut(train_df.Number_Weeks_Used, bins= 5)
train_df['Quit_Bins']= pd.cut(train_df.Number_Weeks_Quit, bins= 5)

train_df.drop(['Insect_Bins', 'Dose_Bins', 'Use_Bins', 'Quit_Bins'], axis= 1, inplace= True)
train_df.head(3)'''

In [ ]:
comb_df['Insect_Bins']= pd.cut(comb_df.Estimated_Insects_Count, bins= 7)
comb_df['Dose_Bins']= pd.cut(comb_df.Number_Doses_Week, bins= 5)
comb_df['Use_Bins']= pd.cut(comb_df.Number_Weeks_Used, bins= 5)
comb_df['Quit_Bins']= pd.cut(comb_df.Number_Weeks_Quit, bins= 5)

In [ ]:
comb_df.head(3)

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,Org_Data
0,F00000001,188,1,0,0,0,0.0,0,0,0.0,Train
1,F00000003,209,1,0,0,0,0.0,0,1,1.0,Train
2,F00000004,257,1,0,0,0,0.0,0,1,1.0,Train


In [ ]:
Encoding('Insect_Bins')
Encoding('Dose_Bins')
Encoding('Use_Bins')
Encoding('Quit_Bins')

## Model Fitting

In [ ]:
train_f= comb_df[comb_df.Org_Data== 'Train'].reset_index()
test_f= comb_df[comb_df.Org_Data== 'Test'].reset_index()

train_f.drop("index", axis= 1, inplace= True)
test_f.drop("index", axis= 1, inplace= True)

In [ ]:
train_f.columns

Index(['ID', 'Estimated_Insects_Count', 'Crop_Type', 'Soil_Type',
       'Pesticide_Use_Category', 'Number_Doses_Week', 'Number_Weeks_Used',
       'Number_Weeks_Quit', 'Season', 'Crop_Damage', 'Org_Data', 'PestWt',
       'Pesticide_Use_Amt', 'Crop_Seasn_Soil'],
      dtype='object')

In [ ]:
#X = train_f.drop(["ID", "Crop_Damage", "Estimated_Insects_Count", "Number_Doses_Week", "Number_Weeks_Used", 
#"Number_Weeks_Quit", "Org_Data"], axis = 1)

X = train_f.drop(["ID", "Crop_Damage", "Org_Data", "PestWt", "Crop_Type", "Soil_Type",
                  "Number_Doses_Week", "Number_Weeks_Used", "Season", "PestWt"], axis = 1)
Y = train_df['Crop_Damage']

In [ ]:
X.head(3)

,Estimated_Insects_Count,Pesticide_Use_Category,Number_Weeks_Quit,Pesticide_Use_Amt,Crop_Seasn_Soil
0,188,0,0,0.0,0
1,209,0,0,0.0,0
2,257,0,0,0.0,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0,stratify = Y)
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 123)

In [ ]:
# describes info about train and test set 
print("Number transactions X_train dataset: ", X_train.shape) 
print("Number transactions y_train dataset: ", y_train.shape) 
print("Number transactions X_test dataset: ", X_test.shape) 
print("Number transactions y_test dataset: ", y_test.shape)

Number transactions X_train dataset:  (62200, 5)
Number transactions y_train dataset:  (62200,)
Number transactions X_test dataset:  (26658, 5)
Number transactions y_test dataset:  (26658,)


In [ ]:
def Training_Report(Model,x_train= X_train, x_test= X_test, Y_train= y_train, Y_test= y_test, label = 'C'):
    
    Model.fit(x_train, Y_train.ravel()) 
    predictions = Model.predict(x_test)
    
    print('\n',label)

    # print classification report 
    print('\n Classification Report:\n',classification_report(Y_test, predictions)) 
    
    #Confusion Matrix
    conf_mat = confusion_matrix(y_true=Y_test, y_pred=predictions)

    print('\n Confusion matrix:\n', conf_mat)
    
    print('\n Accuracy Score:\n', accuracy_score(Y_test,predictions))
    #print('\n ROC_AUC score:\n',roc_auc_score(Y_test,predictions, multi_class= 'ovr'))

In [ ]:
#Decision Tree
DT = DecisionTreeClassifier(random_state = 0) 

#Random Forest
RF = RandomForestClassifier(random_state = 0,n_jobs = -1, class_weight= weights)

# Gradient Boost
GB = GradientBoostingClassifier(learning_rate=0.01,random_state=0,n_estimators=1000 )


In [ ]:
Training_Report(DT, label = 'Decision Tree')


 Decision Tree

 Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.89      0.87     22272
           1       0.26      0.24      0.25      3692
           2       0.05      0.03      0.04       694

    accuracy                           0.77     26658
   macro avg       0.39      0.38      0.39     26658
weighted avg       0.76      0.77      0.77     26658


 Confusion matrix:
 [[19754  2235   283]
 [ 2724   872    96]
 [  464   209    21]]

 Accuracy Score:
 0.7745142171205642


In [ ]:
Training_Report(RF, label = 'Random Forest')


 Random Forest

 Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.88      0.88     22272
           1       0.28      0.25      0.26      3692
           2       0.10      0.17      0.13       694

    accuracy                           0.77     26658
   macro avg       0.42      0.43      0.42     26658
weighted avg       0.78      0.77      0.77     26658


 Confusion matrix:
 [[19525  2147   600]
 [ 2341   921   430]
 [  369   205   120]]

 Accuracy Score:
 0.771475729612124


In [ ]:
Training_Report(GB, label = 'Gradient Boosting')


 Gradient Boosting

 Classification Report:
               precision    recall  f1-score   support

           0       0.84      1.00      0.91     22272
           1       0.68      0.05      0.09      3692
           2       0.00      0.00      0.00       694

    accuracy                           0.84     26658
   macro avg       0.51      0.35      0.34     26658
weighted avg       0.80      0.84      0.78     26658


 Confusion matrix:
 [[22235    37     0]
 [ 3507   184     1]
 [  646    48     0]]

 Accuracy Score:
 0.8409858203916273


In [ ]:
def get_class_weights(y):
    counter = Counter(y)
    majority = max(counter.values())
    print(majority)
    #return  {cls: round(float(majority)/float(count), 2) for cls, count in counter.items()}
    return  {cls: round(sum(counter.values())/float(count), 2) for cls, count in counter.items()}

weights = get_class_weights(Y)
print(weights)

74238
{0: 1.2, 1: 7.22, 2: 38.42}


In [ ]:
LGB = LGBMClassifier(n_estimators=550, learning_rate=0.03, objective= 'multiclass', num_class= 3,
                     random_state=1, colsample_bytree=0.5, reg_alpha=2, reg_lambda=2, is_unbalance= True )

LGB.fit(X_train, y_train, eval_set=[ (X_test, y_test)], verbose=50, eval_metric = 'multi_error' )

[50]	valid_0's multi_error: 0.159389	valid_0's multi_logloss: 0.460084
[100]	valid_0's multi_error: 0.159127	valid_0's multi_logloss: 0.444167
[150]	valid_0's multi_error: 0.158977	valid_0's multi_logloss: 0.438232
[200]	valid_0's multi_error: 0.158752	valid_0's multi_logloss: 0.435448
[250]	valid_0's multi_error: 0.158752	valid_0's multi_logloss: 0.434118
[300]	valid_0's multi_error: 0.159052	valid_0's multi_logloss: 0.433461
[350]	valid_0's multi_error: 0.158864	valid_0's multi_logloss: 0.433086
[400]	valid_0's multi_error: 0.158339	valid_0's multi_logloss: 0.432858
[450]	valid_0's multi_error: 0.158264	valid_0's multi_logloss: 0.432716
[500]	valid_0's multi_error: 0.158076	valid_0's multi_logloss: 0.432617
[550]	valid_0's multi_error: 0.157814	valid_0's multi_logloss: 0.432624


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', is_unbalance=True, learning_rate=0.03,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=550, n_jobs=-1, num_class=3,
               num_leaves=31, objective='multiclass', random_state=1,
               reg_alpha=2, reg_lambda=2, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [ ]:
xgb_param= {'objective': 'multi: softmax',
            'eta': 0.01 , 'subsample': 0.8 ,
            'eval_metric' : 'mlogloss', 'scale_pos_weight': weights, 'min_child_weight': 5, 'max_depth': 6 , 'colsample_bytree': 0.7,
            'silent':1,'seed': 0, 'seed': 123}
            

XGB= xgb.XGBClassifier(**xgb_param )

In [ ]:
Training_Report(XGB, label= 'XGB')


 XGB

 Classification Report:
               precision    recall  f1-score   support

           0       0.85      1.00      0.92     22272
           1       0.60      0.08      0.14      3692
           2       0.00      0.00      0.00       694

    accuracy                           0.84     26658
   macro avg       0.48      0.36      0.35     26658
weighted avg       0.79      0.84      0.78     26658


 Confusion matrix:
 [[22173    99     0]
 [ 3409   282     1]
 [  608    86     0]]

 Accuracy Score:
 0.8423362592842674


In [ ]:
XGB.fit(X, Y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.01,
              eval_metric='mlogloss', gamma=0, learning_rate=0.1,
              max_delta_step=0, max_depth=6, min_child_weight=5, missing=None,
              n_estimators=100, n_jobs=1, nthread=None,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight={0: 1.2, 1: 7.22, 2: 38.42},
              seed=123, silent=1, subsample=0.8, verbosity=1)

In [ ]:
from sklearn.ensemble import VotingClassifier
model1= VotingClassifier(estimators= [('lgb', LGB), ('xgb', XGB)])


In [ ]:
model.fit(X, Y)

VotingClassifier(estimators=[('lgb',
                              LGBMClassifier(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=0.5,
                                             importance_type='split',
                                             is_unbalance=True,
                                             learning_rate=0.03, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=550, n_jobs=-1,
                                             num_class=3, num_leaves=31,
                                             objective='multiclass',
                                             random_state=1, reg_alpha=2...
                                            eval_metric='ml

In [ ]:
!pip install catboost

     |████████████████████████████████| 64.8MB 60kB/s 


In [ ]:
from catboost import CatBoostClassifier

In [ ]:
model= CatBoostClassifier()

In [ ]:
model.fit(X, Y)

Learning rate set to 0.099473
0:	learn: 0.9818414	total: 103ms	remaining: 1m 43s
1:	learn: 0.8925196	total: 146ms	remaining: 1m 12s
2:	learn: 0.8228737	total: 192ms	remaining: 1m 3s
3:	learn: 0.7679528	total: 226ms	remaining: 56.2s
4:	learn: 0.7205628	total: 268ms	remaining: 53.2s
5:	learn: 0.6814398	total: 318ms	remaining: 52.7s
6:	learn: 0.6483006	total: 365ms	remaining: 51.8s
7:	learn: 0.6204326	total: 414ms	remaining: 51.3s
8:	learn: 0.5969219	total: 456ms	remaining: 50.3s
9:	learn: 0.5769722	total: 494ms	remaining: 48.9s
10:	learn: 0.5589650	total: 536ms	remaining: 48.2s
11:	learn: 0.5436843	total: 581ms	remaining: 47.8s
12:	learn: 0.5303260	total: 625ms	remaining: 47.5s
13:	learn: 0.5188975	total: 670ms	remaining: 47.2s
14:	learn: 0.5093436	total: 712ms	remaining: 46.8s
15:	learn: 0.5007767	total: 757ms	remaining: 46.5s
16:	learn: 0.4929279	total: 799ms	remaining: 46.2s
17:	learn: 0.4863038	total: 844ms	remaining: 46s
18:	learn: 0.4803903	total: 887ms	remaining: 45.8s
19:	learn: 

## Prediction

In [ ]:
#pred = GB.predict(test_f[X.columns])
pred = model.predict(test_f[X.columns])

In [ ]:
test_f["Crop_Damage"]= pred
df_subm = test_f[['ID', 'Crop_Damage']]
df_subm.head()

,ID,Crop_Damage
0,F00000002,1
1,F00000007,1
2,F00000011,1
3,F00000013,1
4,F00000014,1


In [ ]:
df_subm.to_csv('/content/drive/My Drive/Agri_AV/Submission_13_team_Loop.csv',index = False)

In [ ]:
df_subm.Crop_Damage.value_counts()

0    57677
1     1626
2        7
Name: Crop_Damage, dtype: int64